# Pandas Homework Part 2

`polars` version

In [1]:
import pandas as pd
import polars as pl

In [2]:
columns = ['prev', 'curr', 'type', 'n']

In [4]:
%%time
for country in ["de", "pl"]:
    dfl = pl.read_csv(f"../data/wikipedia/clickstream-{country}wiki-2022-03.tsv.gz",sep="\t", has_header=False, new_columns=columns, quote_char=None)
    s = (
        dfl.lazy()
        .filter(pl.col("type") =="external")
        .groupby("curr")
        .agg(pl.col("n").sum().alias("total"))
        .sort("total",reverse=True)
        .collect()[0, 'curr']
    )
    print(country, s)

de Ukraine
pl Ukraina
CPU times: user 6.5 s, sys: 1.83 s, total: 8.32 s
Wall time: 2.54 s


In [5]:
badges = pd.read_xml("../data/travel/travel.stackexchange.com/Badges.xml")
posts = pd.read_xml("../data/travel/travel.stackexchange.com/Posts.xml", parser='etree')
tags =  pd.read_xml("../data/travel/travel.stackexchange.com/Tags.xml", parser='etree')
users =  pd.read_xml("../data/travel/travel.stackexchange.com/Users.xml", parser='etree')
votes =  pd.read_xml("../data/travel/travel.stackexchange.com/Votes.xml", parser='etree')

In [6]:
badges_pl = pl.from_pandas(badges)
posts_pl = pl.from_pandas(posts)
tags_pl = pl.from_pandas(tags)
votes_pl = pl.from_pandas(votes)
users_pl = pl.from_pandas(users)
posts_pl['OwnerUserId'] = posts_pl['OwnerUserId'].cast(int)

In [7]:
wiki_pl = pl.read_csv(f"../data/wikipedia/clickstream-enwiki-2022-03.tsv.gz", sep="\t", has_header=False, new_columns=columns, quote_char="")

In [8]:
%%time 
# 3, 4
tid = (
    badges_pl
    .join(users_pl, left_on="UserId", right_on="Id", how='left')
    .groupby(["UserId", "DisplayName"])
    .agg([pl.count().alias("NBadges")])
    .sort("NBadges", reverse=True)
    .head(1)
    [0, 'UserId']
)
top_user =(
    users_pl
    .filter(pl.col("Id") == tid)
    .select(['DisplayName', 'Location'])
)
top_user

CPU times: user 393 ms, sys: 94.4 ms, total: 488 ms
Wall time: 108 ms


DisplayName,Location
str,str
"""Mark Mayo""","""Christchurch, New Zealand"""


In [9]:
%%time 
# 3, 4 lazy evaluation
(
    badges_pl.lazy()
    .join(users_pl.lazy(), left_on="UserId", right_on="Id", how='left')
    .groupby(["UserId", "DisplayName"])
    .agg([pl.count().alias("NBadges")])
    .sort("NBadges", reverse=True)
    .head(1)
    .collect()[0, ['UserId']]
)
top_user = (
    users_pl
    .lazy()
    .filter(pl.col("Id") == tid)
    .select(['DisplayName', 'Location'])
    .collect()
)
top_user

CPU times: user 170 ms, sys: 38.5 ms, total: 208 ms
Wall time: 44.6 ms


DisplayName,Location
str,str
"""Mark Mayo""","""Christchurch, New Zealand"""


In [10]:
%%time
# 5
city = top_user['Location'][0].split(", ")[0]
(
    wiki_pl
    .filter(pl.col('curr') == city)
    .select(pl.col('n').sum())
)

CPU times: user 93 ms, sys: 1.29 ms, total: 94.3 ms
Wall time: 78.3 ms


n
i64
25804


In [11]:
%%time
# 6, 7
res = (
    posts_pl
    .select(
        pl.col('Body')
            .str.replace_all("<.*?>", "")
            .str.replace_all("\n", " ")
            .str.split(" ")
            .explode()
            .str.to_lowercase()
            .alias("Words")
    )
    .select(
        pl.col("Words")
            .filter(pl.col("Words").str.lengths() > 7)
            .value_counts()
    ).unnest("Words")
    .head(1)
    
)
res[0, 'Words'], wiki_pl.filter(pl.col('curr') == res[0, 'Words'].capitalize())['n'].sum()

CPU times: user 2.91 s, sys: 427 ms, total: 3.34 s
Wall time: 3.28 s


('passport', 31631)

In [12]:
%%time
# 8, 9
upvotes_pl = (
    votes_pl
    .lazy()
    .filter(pl.col("VoteTypeId") == 2)
    .groupby("PostId")
    .agg(pl.count().alias("UpVotes"))
)

downvotes_pl = (
    votes_pl
    .lazy()
    .filter(pl.col("VoteTypeId") == 3)
    .groupby("PostId")
    .agg(pl.count().alias("DownVotes"))
)

(
    posts_pl.lazy()
    .join(upvotes_pl, left_on="Id", right_on="PostId", how='left')
    .join(downvotes_pl, left_on="Id", right_on="PostId", how='left')
    .with_columns(
        [
            pl.col("UpVotes").fill_null(0),
            pl.col("DownVotes").fill_null(0),
        ]
    )
    .with_column(
        (pl.col('UpVotes') - pl.col('DownVotes')).alias('UpVoteRatio')
    )
    .join(users_pl.lazy(), left_on="OwnerUserId", right_on="Id")
    .sort('UpVoteRatio', reverse=True)
    .collect()[0, ['DisplayName', 'UpVoteRatio']]
)


CPU times: user 1.24 s, sys: 754 ms, total: 1.99 s
Wall time: 469 ms


DisplayName,UpVoteRatio
str,i64
"""Andrew Lazarus""",547


In [13]:
%%time 
# 10
votes_agg = (
     votes_pl
    .with_column(
        pl.col('CreationDate').str.strptime(pl.Datetime)
    )
    .groupby([
        pl.col('CreationDate').dt.year().alias("Year"),
        pl.col('CreationDate').dt.month().alias("Month")
    ])
    .agg(pl.count().alias("NVotes"))
)

votes_agg.filter(pl.col("NVotes") == pl.col("NVotes").max())

CPU times: user 591 ms, sys: 3.48 ms, total: 595 ms
Wall time: 518 ms


Year,Month,NVotes
i32,u32,u32
2016,8,19591


In [14]:
%%time
# 11
(
    votes_agg
    .sort(["Year", "Month"])
    .select([
        "Year",
        "Month",
        pl.col("NVotes").cast(int).diff().alias("NVotesDiff")
    ])
    .filter(pl.col("NVotesDiff") == pl.col("NVotesDiff").min())
)

CPU times: user 8.34 ms, sys: 1.12 ms, total: 9.46 ms
Wall time: 2.72 ms


Year,Month,NVotesDiff
i32,u32,i64
2015,10,-6201


In [15]:
%%time
# 12
(
    posts_pl.lazy().join(users_pl.lazy(), left_on="OwnerUserId", right_on="Id", how='left')
    .filter(
        pl.col("Location").str.contains("Poland") | 
        pl.col("Location").str.contains("Polska")
    )
    .select([
        pl.col('Tags')
            .str.replace(r"^<", "")
            .str.replace(r">$", "")
            .str.split("><")
            .drop_nulls()
            .explode()
            .value_counts()
    ])
    .unnest("Tags")
    .head(1)
    .collect()
)


CPU times: user 42.3 ms, sys: 15.4 ms, total: 57.7 ms
Wall time: 21.6 ms


Tags,counts
str,u32
"""air-travel""",34
